In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, median_absolute_error

%matplotlib inline

# Offline Policy Evaluation
Based on: https://github.com/banditml/offline-policy-evaluation

In [207]:
def current_notification_policy(last_active):
    """
    Send a notification with 0.9 probability if a user was
    last active more than 1 days ago.

    0.1 probability for exploration.
    """
    
    epsilon = 0.1
    return 1 - epsilon if last_active > 1 else epsilon

In [275]:
def generate_sample(policy):
    """
    Generate an action using a `policy` and compute the reward
    based on the action and user response.
    
    reward = 10 if a user logged in after receiveing _no_ notification
    reward = 7 if a user logged in after being notified (we penalize notifications)
    reward = 0 if not logged in regardless of the action
    """
    
    last_active = int(round(np.random.exponential(3)))
    send_prob = policy(last_active)
    action = "send" if send_prob > np.random.rand() else "dont_send"
    
    active_after_action = np.random.rand() < 0.78
    match (active_after_action, action):
        case (True, "dont_send"):
            reward = 10
        case (True, "send"):
            reward = 7
        case (False, _):
            reward = 0
    
    return {
        "last_active": last_active,
        "send_prob": send_prob,
        "active_after_action": active_after_action,
        "action": action,
        "reward": reward
    }


def generate_samples(policy, n=1000):
    np.random.seed()
    return pd.DataFrame(generate_sample(policy) for _ in range(n))

In [276]:
data = generate_samples(current_notification_policy)

In [164]:
test = pd.DataFrame(
    {
        "last_active": [0, 0, 1, 2, 5, 7, 8, 10, 14],
        "send_prob": [.1, .1, .1, .9, .9, .9, .9, .9, .9],
        "action": ["dont_send"] * 4 + ["send", "dont_send"] * 2 + ["send"],
        "reward": [10, 10, 10, 10, 7, 0, 7, 0, 7]
    }
)

### Inverse Propensity Scoring (IPS)

IPS computes an action distribution mismatch between the new policy and the
logged policy that generated the historical data. It is defined as

$$\hat{V} = \frac{1}{N}\sum_{k=1}^{N}\frac{\nu(a_k \mid x_k)}{\hat{\mu}_k(a_k \mid x_k)}r_k$$

where $\hat{V}$ is the expected average reward per decision under the new policy, $N$ is the
number of logged historical samples, $\nu(a_k \mid x_k)$ is the probability that the new policy
takes the logged action $a_k$ given the logged context $x_k$, $\hat{\mu}_k(a_k \mid x_k)$ is the
probability that your logged policy takes the logged action $a_k$ given the logged context $x_k$,
and $r_k$ is the logged reward for the given historical sample $k$.

For samples with high reward $r_k$, we want $\nu$ to be much larger than $\mu_k$. This means that
our new policy works better than the logged policy.

In [208]:
def new_good_notification_policy(last_active):

    epsilon = 0.1
    return 1 - epsilon if last_active > 5 else epsilon

def new_bad_notification_policy(last_active):

    epsilon = 0.1
    return 1 - epsilon if last_active < 5 else epsilon

In [302]:
def sample_ips(sample, policy, no_reward_weight=False):
    """
    Computes IPS for one sample.
    """

    send_prob = sample["send_prob"]
    # last_active := our context
    last_active = sample["last_active"]

    if sample["action"] == "send":
        mu = send_prob
        nu = policy(last_active)
    else:
        mu = 1 - send_prob
        nu = 1 - policy(last_active)

    return (nu / mu) * (1 if no_reward_weight else sample["reward"])


def inverse_propensity_scoring(data, policy):
    """
    Computes total IPS.
    """
    
    N = len(data)
    _sample_ips = partial(sample_ips, policy=policy, no_reward_weight=False)
    return data.apply(_sample_ips, axis=1).sum() / N

In [303]:
def evaluate(scoring, data):
    
    policies = {
        "current": current_notification_policy,
        "new_good": new_good_notification_policy,
        "new_bad": new_bad_notification_policy
    }
    
    for name, policy in policies.items():
        print(name, f"score={scoring(data, policy):.2f}")

In [304]:
# inverse_propensity_scoring(data, current_notification_policy)
# equals to data["reward"].sum() / len(data)
# because every nu / mu = 1

evaluate(inverse_propensity_scoring, test)

current score=6.78
new_good score=14.98
new_bad score=1.74


### Direct Method (DM)

DM uses a model predicting the reward $\hat{r}$ based on a action and context.

$$\hat{V} = \frac{1}{N} \sum_{k=1}^{N} \sum_{a \in A} \nu(a \mid x_k) \hat{r}(x_k, a)$$

where $\hat{V}$ is the expected average reward per decision under the new policy, $N$ is the
number of logged historical samples, $\nu(a \mid x_k)$ is the probability that the new policy
takes an action $a$ given the logged context $x_k$, $\hat{r}(x_k, a)$ is the predicted
reward using the context $x_k$ of sample $k$ an action $a$ and $A$ is the set of all posible actions.

We need a model beacuse for each logged context $x_k$, we want to predict the reward of every possible action
$a \in A$ and we only have one action $a_k$ logged.

For an action with high predicted reward $\hat{r}(x_k, a)$, we want large probability $\nu(a \mid x_k)$ of the action.

In [292]:
def reward_model_factory(data):    
    # handle categorical "action" feature
    X = pd.get_dummies(
        data=data[["last_active", "action"]],
        columns=["action"], 
        drop_first=True
    ).values
    y = data["reward"].values
    
    model = GradientBoostingRegressor(random_state=0)
    
    model.fit(X, y)
    print(f"MSE={mean_squared_error(y, model.predict(X)):.2f}")
    print(f"MAE={median_absolute_error(y, model.predict(X)):.2f}")
    return model


def sample_dm(sample, policy, reward_model):
    """
    Computes DM for one sample.
    """
    last_active = sample["last_active"]

    # r_hat for both actions (send, dont_send), same context
    # 1 := send, 0 := dont_send
    r_hats = reward_model.predict(
        [
            [last_active, 1],
            [last_active, 0]
        ]
    )

    send_prob = policy(last_active)
    # nu for both actions, same context (last_active is same for both)
    nus = np.array([send_prob, 1 - send_prob])

    # inner sum
    return np.dot(nus, r_hats)


def direct_method(data, policy, reward_model):
    """
    Computes total DM.
    """

    # outer sum    
    N = len(data)
    _sample_dm = partial(sample_dm, policy=policy, reward_model=reward_model)
    return data.apply(_sample_dm, axis=1).sum() / N

In [293]:
reward_model = reward_model_factory(data)

MSE=11.82
MAE=2.23


In [294]:
evaluate(partial(direct_method, reward_model=reward_model), test)

current score=6.21
new_good score=6.74
new_bad score=5.73


### Doubly Robust Method (DRM)

Doubly robust method combines DM and IPS and is a strict improvement over both methos.
It is defined as

$$
    \hat{V} = \frac{1}{N} \sum_{k=1}^{N}
    \left(
    \sum_{a \in A} \nu(a \mid x_k) \hat{r}(x_k, a)
    + \frac{\nu(a_k \mid x_k)}{\hat{\mu}_k(a_k \mid x_k)} (r_k - \hat{r}(x_k, a_k))
    \right)
$$

We can see from the formula that the method is using DM and then applies a correction based on IPS.
If the reward predicted by the model sample $k$ is equal to the logged reward $r_k$ then

$$
(r_k - \hat{r}(x_k, a_k)) = 0
$$

and the result is equal to just the direct method.

In [308]:
def sample_drm(sample, policy, reward_model):
    """
    Computes DRM for one sample.
    """
    
    last_active = sample["last_active"]
    is_send = sample["action"] == "send"

    dm = sample_dm(sample, policy, reward_model)
    raw_ips = sample_ips(sample, policy, no_reward_weight=True)
    
    r = sample["reward"]
    r_hat = reward_model.predict([[last_active, int(is_send)]])[0]
    ips_weight = r - r_hat
    
    return dm + raw_ips * ips_weight


def doubly_robust_method(data, policy, reward_model):
    """
    Computes total DRM.
    """
    
    # outer sum    
    N = len(data)
    _sample_drm = partial(sample_drm, policy=policy, reward_model=reward_model)
    return data.apply(_sample_drm, axis=1).sum() / N

In [309]:
evaluate(partial(doubly_robust_method, reward_model=reward_model), test)

current score=6.26
new_good score=8.46
new_bad score=-7.47
